# finished code

In [1]:
import argparse
import ipaddress
import os
import networkx as nx

from networkx.algorithms.cluster import average_clustering
from networkx.algorithms.components import *
from networkx.algorithms.dag import (dag_longest_path_length,
									 is_directed_acyclic_graph)
import networkx.algorithms.approximation as approx
from networkx.algorithms.connectivity import node_connectivity, edge_connectivity

from modules import Rule
from modules_hit import Rule_hit
from utils import construct_graph
import numpy as np
import random


def load_ruleset(fname, except_zero = True, random_priority = 0):
	""" Load ruleset from ClassBench filter file """
	"""
	(expect_zero = True): due to zero nodes(0.0.0.0/n) causes huge computing resource, you can exclude all zero nodes here.
	(random_priorit = n): from 0 to n. if n = 0, use the sequence of the ruleset as each rule's priority.
	"""
	ruleset = []
	with open(fname, 'r') as f:
		for n, line in enumerate(f):
			# LINE FORMAT
			# @sip_network dip_network sp_low : sp_high dp_low: dp_high protocal/protocol_mask xxx/xxx
			tok = line.strip().split('\t')
			rule = Rule_hit()
			sip = ipaddress.ip_network(tok[0][1:])
			dip = ipaddress.ip_network(tok[1])
			sp = tok[2].split(':')
			dp = tok[3].split(':')
			protocol = tok[4].split('/')
			
			if except_zero:
				if int(sip[0]) == 0 or int(dip[0]) == 0:
					continue
			#sip[0] : given a IP/mask, upper bound of IP address
			#sip[-1] : given a IP/mask, lower bound of IP address
			rule.sip_low = int(sip[0])
			rule.sip_high = int(sip[-1])
			rule.dip_low = int(dip[0])
			rule.dip_high = int(dip[-1])
			rule.sp_low, rule.sp_high = int(sp[0]), int(sp[1])
			rule.dp_low, rule.dp_high = int(dp[0]), int(dp[1])
			rule.protocol_val, rule.protocol_mask = int(protocol[0], 16), int(protocol[1], 16)

			if random_priority:
				rule.priority = int(random.randint(0, random_priority))
			else:
				rule.priority = n
			ruleset.append(rule)
	return ruleset

def all_path_lenth(graph):
	paths_len = []
	for idx_s in [idx for idx,i in enumerate(list(dict(G.in_degree()).values())) if (i == 0 and G.out_degree(idx) != 0)]:
		for idx_d in [idx for idx,i in enumerate(list(dict(G.out_degree()).values())) if (i == 0 and G.in_degree(idx) != 0)]:
			if nx.has_path(G,idx_s, idx_d):
				l = list(nx.all_simple_paths(G, source=idx_s, target=idx_d))
				paths_len.append(len(l))
			else:
				continue
	return np.array(paths_len)

def rule_intersect(rule_a, rule_b):
    """ Decide if two rules intersect """
    ab_sip_split = get_split(rule_a.sip_low, rule_a.sip_high, rule_b.sip_low, rule_b.sip_high)
    if not ab_sip_split:
        return False

    ab_dip_split = get_split(rule_a.dip_low, rule_a.dip_high, rule_b.dip_low, rule_b.dip_high)
    if not ab_dip_split:
        return False

    ab_sp_split = get_split(rule_a.sp_low, rule_a.sp_high, rule_b.sp_low, rule_b.sp_high)
    if not ab_sp_split:
        return False

    ab_dp_split = get_split(rule_a.dp_low, rule_a.dp_high, rule_b.dp_low, rule_b.dp_high)
    if not ab_dp_split:
        return False
    
    temp_ruleset_a = split_to_rule(ab_sip_split[0], ab_dip_split[0], ab_sp_split[0], ab_dp_split[0], rule_a.hit_time, rule_a.protocol_val, rule_a.protocol_mask, rule_a.priority)
    # the a rule will be divided into many rules, temp_ruleset_a is a list contain them
    temp_ruleset_b = split_to_rule(ab_sip_split[1], ab_dip_split[1], ab_sp_split[1], ab_dp_split[1], rule_b.hit_time, rule_b.protocol_val, rule_b.protocol_mask, rule_b.priority)
    temp_ruleset = merge2cube(temp_ruleset_a, temp_ruleset_b)
    return temp_ruleset

def get_split(a_low, a_high, b_low, b_high):
    # receives two ranges, if they has intersect, function returns the intersection (a_split, b_split)
    # each split at most will have three parts
    # else return False
    """warning: caution of (0,4)(4,6)"""
    if (a_high >= b_low and b_high >= a_low):
        temp_list = [a_low, a_high, b_low, b_high]
        temp_list.sort()
        a_split = split_range([a_low, a_high], [temp_list[1], temp_list[2]])
        b_split = split_range([b_low, b_high], [temp_list[1], temp_list[2]])
        return (a_split, b_split)
    else:
        return False
    
def split_range(mother_range, child_range):
    """receives two lists: mother_range[low, high], child_range[low, high]"""
    """return a list that contain 1, 2 or 3 tuples"""
    """WARNING: only mother and child counts, so make sure of it before call the function"""
    range_split = [(child_range[0], child_range[1])]
    if mother_range[1] > child_range[1]:
        range_split.append((child_range[1]+1, mother_range[1]))
    if mother_range[0] < child_range[0]:
        range_split.insert(0,(mother_range[0], child_range[0]-1))
    return range_split

def same_rule(rule_a, rule_b):
    """only counts sip dip sp dp"""
    if ((rule_a.sip_low == rule_b.sip_low) and (rule_a.sip_high == rule_b.sip_high)):
        if ((rule_a.dip_low == rule_b.dip_low) and (rule_a.dip_high == rule_b.dip_high)):
            if ((rule_a.sp_low == rule_b.sp_low) and (rule_a.sp_high == rule_b.sp_high)):
                if ((rule_a.dp_low == rule_b.dp_low) and (rule_a.dp_high == rule_b.dp_high)):
                    return True
    return False

def mv_dipulation(ruleset):
    """Designed for small scale ruleset: remove duplicated rules. It will change the sequence"""
    temp_ruleset = []
    for index, i in enumerate(ruleset):
        for j in range(index+1, len(ruleset)):
            if same_rule(ruleset[index], ruleset[j]):
                break
        else:
            temp_ruleset.append(ruleset[index])
    else:
        return temp_ruleset
    
def merge2cube(ruleset_a, ruleset_b):
    """the same cube has hit_time * 2"""
    """only designed for 2 intersect rules and returns a merged non-exclution group"""
    temp_ruleset = []
    temp_merge = ruleset_a + ruleset_b
    for i, sing_rule in enumerate(temp_merge):
        for j in range(i+1, len(temp_merge)):
            if same_rule(temp_merge[i], temp_merge[j]):
                temp_merge[j].hit_time += temp_merge[i].hit_time
                break
        else:
            temp_ruleset.append(sing_rule)
    else:
        return temp_ruleset
    
def split_to_rule(sip, dip, sp, dp, hit_time, protocol_val, protocol_mask, priority):
    # each parameter is a [(), (), ()] kind (can be 1, 2 or 3)
    hit_list = []
    for sip_range in sip:
        for dip_range in dip:
            for sp_range in sp:
                for dp_range in dp:
                    hit_list.append(construct_rule_from_ip_port(sip_range, dip_range, sp_range, dp_range, hit_time, protocol_val, protocol_mask, priority))
    return hit_list
                    
def construct_rule_from_ip_port(sip_range, dip_range, sp_range, dp_range, hit_time, protocol_val, protocol_mask, priority):
    temp_rule = Rule_hit()
    temp_rule.sip_low, temp_rule.sip_high = sip_range[0], sip_range[1]
    temp_rule.dip_low, temp_rule.dip_high = dip_range[0], dip_range[1]
    temp_rule.sp_low, temp_rule.sp_high = sp_range[0], sp_range[1]
    temp_rule.dp_low, temp_rule.dp_high = dp_range[0], dp_range[1]
    temp_rule.hit_time = hit_time
    temp_rule.protocol_val = protocol_val
    temp_rule.protocol_mask = protocol_mask
    temp_rule.priority = priority
    return temp_rule

def flush_ruleset(ruleset):
    """split a random ruleset"""
    G = construct_graph(ruleset)
    temp_ruleset = []
    temp_list = list(nx.weakly_connected_components(G))
    for i in temp_list:
        if len(i) == 1:
            for t in i:
                temp_ruleset.append(ruleset[t])
        else:
            temp_child_ruleset = []
            for t in i:
                temp_child_ruleset.append(ruleset[t])
            # temp_child_ruleset don't have non-inter cube
            solve_interarea_once(temp_child_ruleset)
            temp_child_ruleset = flush_ruleset(temp_child_ruleset)
            temp_ruleset.extend(temp_child_ruleset)
    return temp_ruleset

def solve_interarea_once(ruleset):
    # flush a fully intersect area and return a non-inter area
    for index_i, i in enumerate(ruleset):
        for index_j in range(index_i+1, len(ruleset)):
            temp_inter = rule_intersect(i,ruleset[index_j])
            if temp_inter:
                global hit_1_time
                hit_1_time += 1
                del ruleset[index_i]
                del ruleset[index_j-1]
                ruleset.extend(temp_inter)
                return

# huge group workspace

In [ ]:
def construct_ruleset_from_set(node_num_set, ruleset):
    temp_ruleset = []
    for i in node_num_set:
        temp_ruleset.append(ruleset[i])
    return temp_ruleset

In [ ]:
""" Load rules from ClassBench filter file, build a graph,
and print graph statistics """

ruleset = load_ruleset("../data/acl filters/MyFilters10k_1.txt", True)

In [ ]:
%%time
G = construct_graph(ruleset)
list_temp = list(nx.weakly_connected_components(G))

In [ ]:
t = []
for i in list_temp:
    if len(i)>100:
        t.append(construct_ruleset_from_set(i, ruleset))
for i in t:
    print(len(i))

In [ ]:
temp = t[1]
print(temp)

In [ ]:
%%time
hit_1_time = 0
temp_ruleset = flush_ruleset(temp)
print(hit_1_time)

# temp code

In [ ]:
def flush_ruleset_once(ruleset):
    for index_i, i in enumerate(ruleset):
        for index_j in range(index_i+1, len(ruleset)):
            temp_set = rule_intersect(i,ruleset[index_j])
            if temp_set:
                del ruleset[index_i]
                del ruleset[index_j-1]
                ruleset.extend(temp_set)
                return False
    else:
        return True
    
def flush_ruleset(ruleset):
    if not flush_ruleset_once(ruleset):
        flush_ruleset(ruleset)
    else:
        return

In [ ]:
def reflush_ruleset(ruleset_a):
    """ruleset_a flush ruleset_b"""
    temp_ruleset = []
    for coming_rule in ruleset_a:
        if temp_ruleset:
            temp_ruleset = rule_flush_ruleset(coming_rule, temp_ruleset)
            for i in temp_ruleset:
                print(i.sip_low)
            print("wa")
        else:
            temp_ruleset.append(coming_rule)
            for i in temp_ruleset:
                print(i.sip_low)
            print("wa")
    return temp_ruleset

def rule_flush_ruleset(rule, ruleset):
    new_ruleset = []
    for exist_rule in ruleset:
        temp_inter = rule_intersect(rule, exist_rule)
        print(temp_inter)
        print("wa")
        if temp_inter:
            new_ruleset.extend(temp_inter)
        else:
            new_ruleset.append(exist_rule)
        
    return new_ruleset

# developing code

In [2]:
def cut2points(ruleset):
    sip = []
    dip = []
    sp = []
    dp = []
    for i in ruleset:
        sip.append(i.sip_low)
        sip.append(i.sip_high)
        dip.append(i.dip_low)
        dip.append(i.dip_high)
        sp.append(i.sp_low)
        sp.append(i.sp_high)
        dp.append(i.dp_low)
        dp.append(i.dp_high)
    return sorted(set(sip)), sorted(set(dip)), sorted(set(sp)), sorted(set(dp))

def list2mapping(a):
    mapping = {}
    for i,v in enumerate(a):
        mapping[v] = i
    return mapping

def get_point_index(rule,sip_map, dip_map, sp_map, dp_map):
    sip_low = sip_map[rule.sip_low]
    sip_high = sip_map[rule.sip_high]
    dip_low = dip_map[rule.dip_low]
    dip_high = dip_map[rule.dip_high]
    sp_low = sp_map[rule.sp_low]
    sp_high = sp_map[rule.sp_high]
    dp_low = dp_map[rule.dp_low]
    dp_high = dp_map[rule.dp_high]
    return sip_low, sip_high, dip_low, dip_high, sp_low, sp_high, dp_low, dp_high

In [3]:
ruleset = load_ruleset("../data/fw filters/MyFilters10k_{}.txt".format(3), False)
sip, dip, sp, dp = cut2points(ruleset) # returns a set
sip_len = len(sip)
dip_len = len(dip)
sp_len = len(sp)
dp_len = len(dp)

In [4]:
sip_map = list2mapping(sip)
dip_map = list2mapping(dip)
sp_map = list2mapping(sp)
dp_map = list2mapping(dp)

In [5]:
print("{},{},{},{}".format(sip_len, dip_len, sp_len, dp_len))

5484,10321,15,45


In [17]:
temp_1 = np.zeros((sip_len-1, dip_len-1, sp_len-1, dp_len-1), dtype=np.uint8)

In [18]:
temp_1.shape

(5483, 10320, 14, 44)

In [19]:
%%time
for i in ruleset[0:100]:
    sip_low, sip_high, dip_low, dip_high, sp_low, sp_high, dp_low, dp_high = get_point_index(i, sip_map, dip_map, sp_map, dp_map)
    temp_1[sip_low:sip_high, dip_low:dip_high, sp_low:sp_high, dp_low:dp_high] += 1

CPU times: user 657 µs, sys: 51 µs, total: 708 µs
Wall time: 30.1 ms


In [20]:
%%time
temp_1[2334:2337, 4411:4414, 3:5, 13:15]

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 10.5 µs


array([[[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]]], dtype=uint8)

In [ ]:
np.max(temp_1, axis=2)

In [9]:
sip_low, sip_high, dip_low, dip_high, sp_low, sp_high, dp_low, dp_high = get_point_index(ruleset[0],sip_map, dip_map, sp_map, dp_map)

In [10]:
print("{},{},{},{},{},{},{},{}".format(sip_low, sip_high, dip_low, dip_high, sp_low, sp_high, dp_low, dp_high))

2335,2336,4412,4413,4,4,14,14


In [12]:
a = np.zeros((4,4))

In [25]:
a[2:4, 2:3] += 1

In [26]:
a

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 2.],
       [0., 0., 1., 2.]])

In [ ]:
temp_1[2333:2338, 4412:4414, 2:5, 12:15]

In [ ]:
tick = np.zeros((4,4))

In [ ]:
np.add(tick[2:4, 2:4], 1)

In [ ]:
tick

In [ ]:
6%2

In [ ]:
%%time
for i in range(0,10):
    temp_1 = np.add(temp_1[2400:4000, 2000:6000,35:45,1], 1)

In [ ]:
%%time
a = np.ones((4000-2400, 6000-2000, 1, 46-36))

In [ ]:
%%time
np.add(temp_num, a)

In [ ]:
temp[0]

In [ ]:
for i in 

In [ ]:
%%time
count = 0
for i in range(0, 7518420):
    count += 1

In [ ]:
ruleset = load_ruleset("../data/fw filters/MyFilters10k_1.txt", True)
cut_sip_1d(ruleset)

In [ ]:
division = len(sip)*len(dip)*len(sp)*len(dp)

In [ ]:
type(division)

In [ ]:
def find_max_group(G):
    temp = 0
    temp_list = list(nx.weakly_connected_components(G))
    for i in temp_list:
        if len(i)>temp:
            temp = len(i)
    return temp

for strs in ['acl', 'fw', 'ipc']:
    for j in range(1, 11):
        i = 10
        path = "../data/{} filters/MyFilters{}".format(strs,i) + "k_{}.txt".format(j)
        print("{},{}".format(i,j))
        ruleset = load_ruleset(path)
        G = construct_graph(ruleset)
        max_group = find_max_group(G)
        print("----------->{}".format(max_group))

In [ ]:
def flush_ruleset(ruleset):
    """split a random ruleset"""
    G = construct_graph(ruleset)
    temp_ruleset = []
    temp_list = list(nx.weakly_connected_components(G))
    for i in temp_list:
        if len(i) == 1:
            for t in i:
                temp_ruleset.append(ruleset[t])
        else:
            temp_child_ruleset = []
            for t in i:
                temp_child_ruleset.append(ruleset[t])
            print("before len is: {}",format([n.hit_time for n in temp_child_ruleset]))
            for h in temp_child_ruleset:
                print("sip_low: {}, sip_high: {}".format(h.sip_low, h.sip_high))
            solve_fully_intersect_area(temp_child_ruleset) # temp_child_ruleset don't have non-inter cube
            print("after len is: {}",format([n.hit_time for n in temp_child_ruleset]))
            for h in temp_child_ruleset:
                print("sip_low: {}, sip_high: {}".format(h.sip_low, h.sip_high))
            temp_ruleset.extend(temp_child_ruleset)
    return temp_ruleset

def solve_fully_intersect_area_1(ruleset):
    # flush a fully intersect area and return a non-inter area
    for index_i, i in enumerate(ruleset):
        for index_j in range(index_i+1, len(ruleset)):
            temp_inter = rule_intersect(i,ruleset[index_j])
            if temp_inter:
                del ruleset[index_i]
                del ruleset[index_j-1]
                ruleset.extend(temp_inter)
                return

def solve_fully_intersect_area(ruleset):
    solve_fully_intersect_area_1(ruleset)
    temp_ruleset = flush_ruleset(ruleset)
    ruleset = temp_ruleset
    return


In [ ]:
G = construct_graph(ruleset)
list_temp = list(nx.weakly_connected_components(G))

In [ ]:
count = 0
for i in list_temp:
    if len(i) > 1:
        t = i

In [ ]:
temp_ruleset = []
for k in t:
    temp_ruleset.append(ruleset[k])

In [ ]:
len(temp_ruleset)

In [ ]:
%%time
temp_ruleset = flush_ruleset(temp_ruleset)

In [ ]:
%%time
temp_list = flush_ruleset(ruleset)

In [ ]:
len(temp_list)

In [ ]:
GG = construct_graph(temp_list)
list_temp = list(nx.weakly_connected_components(GG))

for i in list_temp:
    if len(i) > 1:
        print("wa")

In [ ]:
temp_list = list(nx.weakly_connected_components(G))

In [ ]:
%%time
temp_ruleset = []
temp_list = list(nx.weakly_connected_components(G))
for i in temp_list:
    if len(i) == 1:
        for t in i:
            temp_ruleset.append(ruleset[t])
    else:
        temp_child_ruleset = []
        for t in i:
            temp_child_ruleset.append(ruleset[t])
        flush_ruleset(temp_child_ruleset)
        temp_ruleset.extend(temp_child_ruleset)

In [ ]:
""" Load rules from ClassBench filter file, build a graph,
and print graph statistics """

ruleset = load_ruleset("../data/acl filters/MyFilters1k_1.txt")
G = construct_graph(ruleset)

In [ ]:
%%time
temp_ruleset = []
temp_list = list(nx.weakly_connected_components(G))
for i in temp_list:
    if len(i) == 1:
        for t in i:
            temp_ruleset.append(ruleset[t])
    else:
        temp_child_ruleset = []
        for t in i:
            temp_child_ruleset.append(ruleset[t])
        flush_ruleset(temp_child_ruleset)
        temp_ruleset.extend(temp_child_ruleset)

In [ ]:
G = construct_graph(temp_ruleset)

In [ ]:
for i in list(nx.weakly_connected_components(G)):
    if len(i) > 1:
        print("fxxk!")

In [ ]:
def ex_group_eat(rule, ex_ruleset):
    """get a exclution ruleset and return a exclution ruleset combined with a new rule"""
    temp_ex_group = ex_ruleset
    for i in ex_ruleset:
        temp_inter = rule_intersect(rule, i)
        if not temp_inter:
            continue
        else:
            ex_ruleset.extend(temp_inter)
            if is_ex_group(temp_ex_group):
                continue
            else:
                ex_ruleset = flush_non_ex_group(ex_ruleset) # ！！
    return ex_ruleset

def flush_non_ex_group(ruleset):
    """receive a non-exgroup and return a exgroup"""
    temp_ex_group = []
    temp_ex_group.append(ruleset[0])
    for index, i in enumerate(ruleset[1:]):
        temp_ex_group = ex_group_eat(i, temp_ex_group)
        
def is_ex_group(random_ruleset):
    """wait for develop"""
    for index, i in enumerate(random_ruleset):
        for j in range(index+1, len(random_ruleset)):
            if rule_intersect(i, random_ruleset[j]):
                return True
    else:
        return False

In [ ]:
def flush_ruleset(ruleset):
    temp_1 = ruleset[0]
    temp_2 = ruleset[1]
    temp_non_inter_ruleset = rule_intersect(tmep_1, temp_2)
    for i in ruleset[2:]:
        temp_non_inter_ruleset = non_inter_ruleset_eat_1(temp_non_inter_ruleset, i)
    return temp_non_inter_ruleset

def non_inter_ruleset_eat_1(ruleset, rule):
    temp_ruleset = ruleset
    if has_inter_relation(ruleset, rule):
        temp_ruleset = flush_ruleset(temp_ruleset)
    else:
        wait()
    return temp_ruleset



# test area

In [ ]:
GG = construct_graph(temp_ruleset)
list_temp = list(nx.weakly_connected_components(GG))

for i in list_temp:
    if len(i) > 1:
        print("wa")

In [ ]:
len(ruleset)

In [ ]:
rule_1 = ruleset[0]
rule_2 = ruleset[1]
rule_3 = ruleset[2]

rule_1.sip_low, rule_1.sip_high = 0, 58888
rule_1.dip_low, rule_1.dip_high = 0, 58888
rule_1.sp_low, rule_1.sp_high = 0, 65535
rule_1.dp_low, rule_1.dp_high = 0, 65535

rule_2.sip_low, rule_2.sip_high = 2000, 2399
rule_2.dip_low, rule_2.dip_high = 0, 58888
rule_2.sp_low, rule_2.sp_high = 0, 65535
rule_2.dp_low, rule_2.dp_high = 0, 65535

rule_3.sip_low, rule_3.sip_high = 2000, 2399
rule_3.dip_low, rule_3.dip_high = 0, 58888
rule_3.sp_low, rule_3.sp_high = 0, 65535
rule_3.dp_low, rule_3.dp_high = 0, 65535

In [ ]:
temp_ruleset = [rule_1, rule_2, rule_3]
for i in temp_ruleset:
    print("sip: {},{}   dip: {},{}   sp: {},{}   dp: {},{}".format(i.sip_low, i.sip_high, i.dip_low, i.dip_high,
                                                                   i.sp_low, i.sp_high, i.dp_low, i.dp_high))

In [ ]:
rule_3

In [ ]:
%%time
temp_ruleset = flush_ruleset(temp_ruleset)

In [ ]:
temp_ruleset

In [ ]:
for i in temp_ruleset:
    print(i.hit_time)

In [ ]:
for i in temp_ruleset:
    print(i)
    print("hit time of this ruleset is: {}".format(i.hit_time))

In [ ]:
%%time
is_ex_group(ruleset)

In [ ]:
def test(list_a):
    del list_a[1]
    del list_a[1]
    return

list_c = [2,1,3,4,4,5]
test(list_c)

In [ ]:
list_c